In [16]:
import pandas as pd
import datetime

In [17]:
! ls

DimDate.csv     DimSite.csv     Fact_table.csv  video_data.csv
DimPlatform.csv DimVideo.csv    Untitled.ipynb  video_data.py


### function to modify platform_name

In [18]:
def platform_name (name):
    if 'IPHONE' in name.upper():
        return 'iPhone'
    elif 'ANDROID' in name.upper():
        return 'Android Phone'
    elif 'IPAD' in name.upper():
        return 'iPad'
    elif 'WEB' in name.upper():
        return 'Web'
    elif 'NEW' in name.upper():
        return 'Desktop'
    else: return 'Unknown'

In [19]:
#Test output
platform_name('App Web')

'Web'

### Import Raw Data

In [20]:
df_raw_data = pd.read_csv("video_data.csv", quotechar='"', error_bad_lines = False)

Skipping line 464716: expected 3 fields, saw 4
Skipping line 464947: expected 3 fields, saw 4
Skipping line 465187: expected 3 fields, saw 4
Skipping line 465442: expected 3 fields, saw 4
Skipping line 465648: expected 3 fields, saw 4



In [21]:
df_raw_data.head()


,DateTime,VideoTitle,events
0,2017-01-11T00:00:31.000Z,App Web|Clips|a-current-affair;2016|William Ty...,"157,120,160,104,162,161,163,164,165,166,171,229"
1,2017-01-11T00:00:53.000Z,news| Shark attacks spearfisherman,"127,157,120,160,104,162,161,171,206"
2,2017-01-11T00:00:21.000Z,news| Shark attacks spearfisherman,"127,157,120,160,104,162,161,163,164,165,166,17..."
3,2017-01-11T00:01:27.000Z,news| Chilean navy films UFO,"157,120,160,104,162,161,170,171,237"
4,2017-01-11T00:00:33.000Z,news| Video shows alleged axe attack at Sydney...,"157,120,160,104,162,161,163,164,165,166,171,229"


In [22]:
df_raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1333653 entries, 0 to 1333652
Data columns (total 3 columns):
DateTime      1333653 non-null object
VideoTitle    1333653 non-null object
events        1333653 non-null object
dtypes: object(3)
memory usage: 30.5+ MB


### Clean the data

#### remove the rows in events which don't have 206

In [23]:
df_raw_data = df_raw_data.\
              loc[lambda df: df.events.\
                  apply(lambda l: '206' in l)]

In [24]:
df_raw_data['DateTime'] = df_raw_data['DateTime'].\
                          apply(lambda t: datetime.\
                                        datetime.\
                                        strptime(t[:-5], \
                                        '%Y-%m-%dT%H:%M:%S'))

df_raw_data['DateTime'] = df_raw_data['DateTime'].\
                          apply(lambda dt: dt.strftime("%d/%m/%Y %H:%M"))

df_raw_data['events'] = df_raw_data['events'].str.split(",")

df_raw_data['Platform'] = df_raw_data['VideoTitle'].\
                          str.split("|").str[0].\
                          apply(lambda pn: platform_name(pn))

df_raw_data['Site'] = df_raw_data['VideoTitle'].\
                        str.split("|").str[0].\
                        apply(lambda si: 'News' if si.upper() == 'NEWS' else 'Unknown')

df_raw_data['Platform'].unique()

df_raw_data['Site'].unique()

df_raw_data['Video'] = df_raw_data['VideoTitle'].\
                       str.split("|").str[-1].str.upper()

In [25]:
df_raw_data.head()

,DateTime,VideoTitle,events,Platform,Site,Video
1,11/01/2017 00:00,news| Shark attacks spearfisherman,"[127, 157, 120, 160, 104, 162, 161, 171, 206]",Desktop,News,SHARK ATTACKS SPEARFISHERMAN
9,11/01/2017 00:03,news| Chilean navy films UFO,"[127, 157, 120, 160, 104, 162, 161, 171, 206]",Desktop,News,CHILEAN NAVY FILMS UFO
10,11/01/2017 00:02,news| Chilean navy films UFO,"[157, 120, 160, 104, 162, 161, 171, 206]",Desktop,News,CHILEAN NAVY FILMS UFO
15,11/01/2017 00:04,news|Exclusive CCTV footage of Minto attack re...,"[157, 120, 160, 104, 162, 161, 171, 206]",Desktop,News,EXCLUSIVE CCTV FOOTAGE OF MINTO ATTACK RELEASED
16,11/01/2017 00:03,news|Blue shark washes up on St Kilda beach,"[127, 157, 120, 160, 104, 162, 161, 171, 206]",Desktop,News,BLUE SHARK WASHES UP ON ST KILDA BEACH


In [26]:
df_raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 498109 entries, 1 to 1333642
Data columns (total 6 columns):
DateTime      498109 non-null object
VideoTitle    498109 non-null object
events        498109 non-null object
Platform      498109 non-null object
Site          498109 non-null object
Video         498109 non-null object
dtypes: object(6)
memory usage: 26.6+ MB


## DimDate

In [27]:
dlt_DimDate = pd.DataFrame()
dlt_DimDate['DateTime'] = df_raw_data['DateTime']
dlt_DimDate.head()

,DateTime
1,11/01/2017 00:00
9,11/01/2017 00:03
10,11/01/2017 00:02
15,11/01/2017 00:04
16,11/01/2017 00:03


### Remove duplicate data

In [28]:
df_DimDate = dlt_DimDate.drop_duplicates(['DateTime']).reset_index()
df_DimDate = df_DimDate.drop(columns='index')

#add surrogate key
df_DimDate['s_key'] = df_DimDate.index

In [29]:
df_DimDate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1440 entries, 0 to 1439
Data columns (total 2 columns):
DateTime    1440 non-null object
s_key       1440 non-null int64
dtypes: int64(1), object(1)
memory usage: 22.6+ KB


In [30]:
df_DimDate.head()

,DateTime,s_key
0,11/01/2017 00:00,0
1,11/01/2017 00:03,1
2,11/01/2017 00:02,2
3,11/01/2017 00:04,3
4,11/01/2017 00:05,4


## Dim Platform

In [38]:
df_DimPlatform = pd.DataFrame()
df_DimPlatform['Platform'] = df_raw_data['Platform'].unique()
df_DimPlatform['S_key'] = df_DimPlatform.index

In [37]:
df_DimPlatform

,Platform,S_key
0,Desktop,0
1,Web,1
2,Android Phone,2
3,iPhone,3
4,iPad,4


## Dim Site

In [36]:
df_DimSite = pd.DataFrame()
df_DimSite['Site'] = df_raw_data['Site'].unique()
df_DimSite['S_key'] = df_DimSite.index


In [35]:
df_DimSite

,Site,S_key
0,News,0
1,Unknown,1


## Dim Video

In [39]:
df_DimVideo = pd.DataFrame()
df_DimVideo['Video'] = df_raw_data['Video'].unique()
df_DimVideo['S_key'] = df_DimVideo.index


In [40]:
df_DimVideo.head()

,Video,S_key
0,SHARK ATTACKS SPEARFISHERMAN,0
1,CHILEAN NAVY FILMS UFO,1
2,EXCLUSIVE CCTV FOOTAGE OF MINTO ATTACK RELEASED,2
3,BLUE SHARK WASHES UP ON ST KILDA BEACH,3
4,CELEBRITY REACTION TO STREEP’S ANTI-TRUMP SPEECH,4


## Fact Table

In [41]:
df_fact_table = pd.DataFrame()

df_fact_table['DateTime_Skey'] = pd.merge(df_raw_data,df_DimDate, on = 'DateTime', how = 'left')['s_key']
df_fact_table['Platform_Skey'] = pd.merge(df_raw_data,df_DimPlatform, on = 'Platform', how = 'left')['S_key']
df_fact_table['Site_Skey'] = pd.merge(df_raw_data,df_DimSite, on = 'Site', how = 'left')['S_key']
df_fact_table['Video_Skey'] = pd.merge(df_raw_data,df_DimVideo, on = 'Video', how = 'left')['S_key']


In [43]:
df_fact_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 498109 entries, 0 to 498108
Data columns (total 4 columns):
DateTime_Skey    498109 non-null int64
Platform_Skey    498109 non-null int64
Site_Skey        498109 non-null int64
Video_Skey       498109 non-null int64
dtypes: int64(4)
memory usage: 19.0 MB


# output dimention table and fact table to csv file 

In [44]:
df_DimDate.to_csv("DimDate.csv", index=False)
df_DimPlatform.to_csv("DimPlatform.csv", index=False)
df_DimSite.to_csv("DimSite.csv", index=False)
df_DimVideo.to_csv("DimVideo.csv", index=False)
df_fact_table.to_csv("Fact_table.csv", index=False)

In [46]:
!ls

DimDate.csv          DimVideo.csv         video_data.csv
DimPlatform.csv      Fact_table.csv       video_data.py
DimSite.csv          Video_data_ETL.ipynb
